In [21]:
"""
To be able to do transfer learning a base model needs to be trained

1. Read a subset of FairFace data set. TODO: Read entire FairFace data set
2. Build base model
    2.1 Choose some model. TODO: Be specific about model chosen
    2.2 Random weights
    2.3 Input size as appropriate
3. Train base model
"""

'\nTo be able to do transfer learning a base model needs to be trained\n\n1. Read a subset of FairFace data set. TODO: Read entire FairFace data set\n2. Build base model\n    2.1 Choose some model. TODO: Be specific about model chosen\n    2.2 Random weights\n    2.3 Input size as appropriate\n3. Train base model\n'

In [22]:
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow as tf
from fairness import fairnessMetrics as fm

In [7]:
!pip install keras-efficientnet

In [54]:
# TODO: Replace with Christophs update code for loading data sets. 
def readImages(folderName, image_size, batch_size):
    train_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="training",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
        labels='inferred'
    )
    val_ds = keras.preprocessing.image_dataset_from_directory(
        folderName,
        validation_split=0.2,
        subset="validation",
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
        labels='inferred'
    )
    return (train_ds, val_ds)

In [70]:
# Build base model
# TODO: Choose appropriate model
base_model = keras.applications.VGG16(
    weights=None, 
    input_shape=(224, 224, 3),
    include_top=True,
    classes=1,
    classifier_activation="sigmoid",
)

In [71]:
# 
metrics_list = ["accuracy",
                tf.keras.metrics.TruePositives(), 
                tf.keras.metrics.TrueNegatives(),
                tf.keras.metrics.FalsePositives(), 
                tf.keras.metrics.FalseNegatives(), 
                fm.TruePositiveRate(),
                fm.TrueNegativeRate(),
                fm.FalsePositiveRate(),
                fm.FalseNegativeRate(),
                fm.PositivePredictedValue(),
                fm.FalseDiscoveryRate(),
                fm.NegativePredictedValue(),
                fm.FalseOmissionRate(),
                fm.BinaryDemographicParityDiff(),
                fm.DemographicParity(),
                fm.BinaryEqualizedOddsDiff(),
                fm.BinaryProportionalParityDiff(),
                fm.ProportionalParity(),
                fm.BinaryPredictiveRateParityDiff(),
                fm.PredictiveRateParity(),
                fm.BinaryAccuracyParityDiff(),
                fm.AccuracyParity(),
                fm.BinaryFalseNegativeRateParityDiff(),
                fm.BinaryFalsePositiveRateParityDiff(),
                fm.BinaryNegativePredictiveRateParityDiff(),
                fm.NegativePredictiveRateParity(),
                fm.BinarySpecificityParityDiff()]

optimizer = keras.optimizers.Adam(learning_rate=1e-3) # TODO: Choose appropriate optimizer and learning_rate

base_model.compile(optimizer=optimizer, 
                   loss="binary_crossentropy", # TODO: look into this
                   #loss="categorical_crossentropy"
                   metrics=metrics_list
)

In [ ]:
"""
Could be implemented?: Batch the data and use caching & prefetching to optimize loading speed
"""

In [ ]:
"""
DATA AUGMENTATION COULD BE ADDED HERE
"""

In [72]:
image_size = (224, 224) # This step should be standardizing, should be able to be changed to what is approptiate
batch_size = 32
data = readImages("FairFace_subset", image_size, batch_size)

Found 200 files belonging to 2 classes.
Using 160 files for training.
Found 200 files belonging to 2 classes.
Using 40 files for validation.


In [73]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_11 (InputLayer)        [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [74]:
epochs = 20
base_model.fit(data[0], epochs=epochs, validation_data=data[1])

Epoch 1/20


2022-11-20 15:33:56.720937: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


5/5 [==============================] - 168s 30s/step - loss: 3125.7681 - accuracy: 0.4938 - true_positives_231: 50.0000 - true_negatives_154: 29.0000 - false_positives_187: 51.0000 - false_negatives_198: 30.0000 - true_positive_rate: 0.6250 - true_negative_rate: 0.3625 - false_positive_rate: 0.6375 - false_negative_rate: 0.3750 - positive_predicted_value: 0.4950 - false_discovery_rate: 0.5050 - negative_predicted_value: 0.4915 - false_omission_rate: 0.5085 - binary_demographic_parity_diff: 42.0000 - demographic_parity: 101.0000 - binary_equalized_odds_diff: -0.2625 - binary_proportional_parity_diff: 0.2625 - proportional_parity: 0.6313 - binary_predictive_rate_parity_diff: 0.0035 - predictive_rate_parity: 0.4950 - binary_accuracy_parity_diff: -0.0125 - accuracy_parity: 0.4938 - false_negative_rate_parity_diff: 0.2625 - false_positive_rate_parity_diff: -0.2625 - binary_negative_predictive_rate_parity_diff: -0.0035 - negative_predictive_rate_parity: 0.4950 - binary_specificity_parity_dif

KeyboardInterrupt: 

In [76]:
base_model.save('/tmp/model')

INFO:tensorflow:Assets written to: /tmp/model/assets
